# Algorithm for enumerating Prx decamer reactions

This notebook performs analyses described in the 

**Manuscript:**

Modelling the decamerisation cycle of PRDX1 and the inhibition-like effect on its peroxidase activity

**Authors:**

C. Barry, C. Pillay, J. Rohwer

**Purpose:**

Enumerates the peroxidase reactions of Prx decamers using the algorithm
described in the manuscript.

Writes the Prx decamer peroxidase reactions without hyperoxidation (filename:`Prx decamer reactions - no hyperoxidation.txt`).

Writes the Prx decamer peroxidase reactions without hyperoxidation and 5 disulphide-bridges (filename:`Prx decamer reactions - no hyperoxidation.txt`).

Writes the Prx decamer peroxidase reactions without hyperoxidation and 5 disulphide-bridges (filename:`Prx decamer reactions - no hyperoxidation 5dsb.txt`).

**Requirements:**

Python libraries (see **Imports** below)


##### Imports

In [ ]:
import os

import numpy as np
from collections import Counter
from sortedcontainers import SortedSet


##### Save directories

In [ ]:
prev_dir = os.path.split(os.getcwd())[0]

mod_dir = os.path.join(prev_dir,"models")
if not os.path.isdir(mod_dir): os.mkdir(mod_dir) # ensure dir exists


## Functions

In [ ]:
def find_leftmost_dimer(dimer):
    """
    Reorients the input dimer according to priority as in "site_in_dimer_priority" using mirror symmetry.
    """
    site_in_dimer_priority = {"SH":4,
                     "SOH":3,
                     "SS":2,
                     "SOOH":1}

    site_1 = dimer.split("_")[0]
    site_2 = dimer.split("_")[1]
    
    site_1_priority = site_in_dimer_priority[site_1]
    site_2_priority = site_in_dimer_priority[site_2]

    if site_1_priority >= site_2_priority:
        output_dimer = f"{site_1}_{site_2}"
    elif site_1_priority < site_2_priority:
        output_dimer = f"{site_2}_{site_1}"
    return output_dimer

def find_leftmost_oligomer(oligomer):
    """
    Reorients the input oligomer according to a score calculated using rotational and planar symmetry.
    """
    dimer_in_oligomer_priority = {"SH_SH":1,
                                  "SH_SOH":2,
                                  "SH_SS":3,
                                  "SH_SOOH":4,
                                  "SOH_SOH":5,
                                  "SOH_SS":6,
                                  "SOH_SOOH":7,
                                  "SS_SS":8,
                                  "SS_SOOH":9, # At this stage no SS_SS is being implimented
                                  "SOOH_SOOH":10
                                 }
    
    # Calculate left score of the current orientation
    full_left_scores = list()
    for i in oligomer:
        # Rotate the oligmer by 1 subunit
        oligomer.insert(0, oligomer.pop(-1))   
        
        # Calculate left score of the current orientation
        species_leftscore = 0
        for idx, dimer in enumerate(oligomer):
            species_leftscore += (idx)*10**(dimer_in_oligomer_priority[dimer])
    
        # Store the left score of the current orientation
        full_left_scores.append(species_leftscore)
        
    # Calculate left score of the flipped orientation
    flipped_oligomer = oligomer[::-1]
    
    flipped_full_left_scores = list()
    for i in flipped_oligomer:
        # Rotate the flipped_oligmer by 1 subunit
        flipped_oligomer.insert(0,flipped_oligomer.pop(-1))   
        
        # Calculate left score of the current orientation
        flipped_species_leftscore = 0
        for idx, dimer in enumerate(flipped_oligomer):
            flipped_species_leftscore += (idx)*10**(dimer_in_oligomer_priority[dimer])
            
        # Store the left score of the current orientation
        flipped_full_left_scores.append(flipped_species_leftscore)

    if max(full_left_scores) >= max(flipped_full_left_scores):
        # Using the left score list, rotate the oligomer to the highest left score orientation
        for i in np.arange(full_left_scores.index(max(full_left_scores))+1):
            oligomer.insert(0,oligomer.pop(-1))
        return oligomer
    
    elif max(flipped_full_left_scores) > max(full_left_scores):
        # Using the left score list, rotate the oligomer to the highest left score orientation
        for i in np.arange(flipped_full_left_scores.index(max(flipped_full_left_scores))+1):
            flipped_oligomer.insert(0,flipped_oligomer.pop(-1))  
        return flipped_oligomer

def find_unique_species(reaction_list):
    """
    Generates a list of unique species in a reaction list for writing the species list in a *.psc model.
    """
    species_list = list()
    dimer_joiner = "__"

    for substrate in reaction_list:
            for product in substrate[1]:
                
                species_list.append(dimer_joiner.join(substrate[0]))
                species_list.append(dimer_joiner.join(product[0]))
            string_species_set = SortedSet(species_list)
            species_set = [i.split(dimer_joiner) for i in string_species_set]
    return species_set
    
def react_oligomer(substrate_oligomer, substrate, product, reaction_type, output_list, recursive=False):
    """
    Generates a list of reactions and products from the input oligomer based on the substrate, product, reaction_type.
    Reorientates products according to mirror, rotational, and planar symmetry.
    Colates the reorientated products and stores statistical factors.
    Can react these products recursively.
    """
    product_species = list()
    for site_idx, site in enumerate(substrate_oligomer):
        if site == substrate: # True means a reaction will take place

            # Find non-reacting site for cooperativity and determine reacted dimer config
            if site_idx % 2 == 0: # Left side of a dimer is reacting
                nonreacting_site = substrate_oligomer[site_idx+1] # Store non-reacting site for cooperativity
                product_dimer = f"{product}_{substrate_oligomer[site_idx+1]}" # Build dimer
                leftmost_product_dimer = find_leftmost_dimer(product_dimer)
                
                # Build reacted oligomer
                product_oligomer = list(substrate_oligomer).copy()
                product_oligomer[site_idx] = leftmost_product_dimer.split("_")[0]
                product_oligomer[site_idx+1] = leftmost_product_dimer.split("_")[1]
                
            elif site_idx % 2 != 0: # Right side of a dimer is reacting
                nonreacting_site = substrate_oligomer[site_idx-1] # Store non-reacting site for cooperativity
                product_dimer = f"{substrate_oligomer[site_idx-1]}_{product}" # Build dimer
                leftmost_product_dimer = find_leftmost_dimer(product_dimer)
                
                # Build reacted oligomer
                product_oligomer = list(substrate_oligomer).copy()
                product_oligomer[site_idx-1] = leftmost_product_dimer.split("_")[0]
                product_oligomer[site_idx] = leftmost_product_dimer.split("_")[1]
                
            rate_constant_name = f"k_Prx_{reaction_type}_{nonreacting_site}_dec"

            product_oligomer = ["_".join([product_oligomer[k],product_oligomer[k+1]]) for k in range(0,len(product_oligomer),2)]
            product_species.append("&&".join(["__".join(find_leftmost_oligomer(product_oligomer)),rate_constant_name])) #Counter in the next line needs a string

    unique_products = Counter(product_species) # Finds the unique products and count of each one

    # Organise the data for output
    products = list()
    for i in unique_products.keys():
        product_only =  i.split("&&")[0]
        rate_constant_name = i.split("&&")[1]
        products.append((tuple(sum([k.split("_") for k in product_only.split("__")],[])),unique_products[i],rate_constant_name))# The "sum" call merges a list of lists
    if bool(products) == True: # Don't append if list is empty i.e when all sites are reacted
        output_list.append((tuple(substrate_oligomer),tuple(products)))
    
    if recursive == True:
        for i in products:
            if substrate in i[0]:
                react_oligomer(i[0], substrate, product, reaction_type, output_list, recursive=True)

def react_disso(cond_species_list):
    """
    Generates a list of dissociation reactions and products from a list of disulphide species.
    """
    disso_reactions = list()
    product_list = list()
    
    for substrate_oligomer in cond_species_list:
        if "SS" in substrate_oligomer:
            product_list = list()
            product_dimers = Counter(["_".join([substrate_oligomer[k],substrate_oligomer[k+1]]) for k in range(0,len(substrate_oligomer),2)])
            
            # Find rate constant name
            site_idx = substrate_oligomer.index("SS")
            if site_idx % 2 == 0:
                nonreacting_site = substrate_oligomer[site_idx+1]
            elif site_idx % 2 != 0: # Right side of a dimer is reacting
                nonreacting_site = substrate_oligomer[site_idx-1] # Store non-reacting site for cooperativity
            rate_constant_name = f"k_Prx_disso_{nonreacting_site}_dec"
            
            for product in product_dimers.keys():
                product_list.append([product_dimers[product],product])

            disso_reactions.append([substrate_oligomer,product_list,rate_constant_name])
    return disso_reactions       

def react_disso(cond_species_list,min_SS=1):
    """
    Generates a list of dissociation reactions and products from a list of disulphide species.
    """
    disso_reactions = list()
    product_list = list()
    
    for substrate_oligomer in cond_species_list:
                
        if substrate_oligomer.count("SS") >= min_SS:
            product_list = list()
            product_dimers = Counter(["_".join([substrate_oligomer[k],substrate_oligomer[k+1]]) for k in range(0,len(substrate_oligomer),2)])
            
            # Find rate constant name
            site_idx = substrate_oligomer.index("SS")
            if site_idx % 2 == 0:
                nonreacting_site = substrate_oligomer[site_idx+1]
            elif site_idx % 2 != 0: # Right side of a dimer is reacting
                nonreacting_site = substrate_oligomer[site_idx-1] # Store non-reacting site for cooperativity
            rate_constant_name = f"k_Prx_disso_{nonreacting_site}_dec"
            
            for product in product_dimers.keys():
                product_list.append([product_dimers[product],product])

            disso_reactions.append([substrate_oligomer,product_list,rate_constant_name])
    return disso_reactions   
    
def write_dec_asso_reactions(f):
    """
    Writes the rate and stoiciometic equation of Prx dimer-decamer dissociation-association reaction.
    """
    f.write("Decamer_association:\n")
    f.write("\t{5}SH_SH = SH_SH__SH_SH__SH_SH__SH_SH__SH_SH")
    f.write("\tk_Prx_asso*SH_SH**5 - k_Prx_dec_disasso*SH_SH__SH_SH__SH_SH__SH_SH__SH_SH")
    f.write("\n")

def get_parameters(parameters_list, reaction_list):
    """
    Grabs a list of reaction rate constants from a list of reactions.
    """
    for reaction in reaction_list:
        parameter = reaction[-1][-1][-1]
        if parameter not in parameters_list:
            parameters_list.append(parameter)
    return SortedSet(parameters_list)

def write_reactions(reaction_list, reaction_name, species_list, reaction_suffix, second_substrate = False):
    """
    Writes a rate and stoiciometic equation for each reaction in the input list for use in a pysces *.psc file.
    """
    f.write(f"# {reaction_name} - decamer\n")
    f.write("\n")
    
    secs_stoic = ""
    secs_rate = ""
    
    if second_substrate:
        secs_stoic = f" + {second_substrate}"
        secs_rate = f"*{second_substrate}"
        
    for reaction in reaction_list:
        substrate_string = dimer_joiner.join(["_".join([reaction[0][k],reaction[0][k+1]]) for k in range(0,len(reaction[0]),2)])
        species_list.append(substrate_string)

        for product in reaction[1]:
            product_string = dimer_joiner.join(["_".join([product[0][k],product[0][k+1]]) for k in range(0,len(product[0]),2)])

            species_list.append(product_string)
            write_reactions.counter += 1
            
            f.write(f"R{str(write_reactions.counter)}_{reaction_suffix}:\n")
            f.write(f"\t{substrate_string}{secs_stoic} > {product_string}\n")
            f.write(f"\t{product[1]}*{product[2]}*{substrate_string}{secs_rate}\n")
            f.write("\n")
    

## Find reactions (no hyperoxidation)

In [ ]:
oligomer_length = 10
subunit = "SH"
original_oligomer = [subunit]*oligomer_length

In [ ]:
# Sulfenilation reactions
substrate = "SH"
product = "SOH"
reaction_type = "sulfe"
sulfe_reactions = list()
react_oligomer(original_oligomer, substrate, product, reaction_type, sulfe_reactions, recursive=True)
set_sulfe_reactions = SortedSet(sulfe_reactions)
unique_sulfe_species = find_unique_species(set_sulfe_reactions)

In [ ]:
print("len sulfe_reactions:",len(sulfe_reactions))
print("len unique_sulfe_species:",len(unique_sulfe_species))

In [ ]:
# Disulfide bridge formation reactions
substrate = "SOH"
product = "SS"
reaction_type = "disulf_form"
disulph_form_reactions = list()

# The unique_sulfi_species contains the unique sulfe species as substrates
for oligomer in unique_sulfe_species: 
    react_oligomer(oligomer, substrate, product, reaction_type, disulph_form_reactions)
set_disulph_form_reactions = SortedSet(disulph_form_reactions)
unique_disulph_form_species = find_unique_species(set_disulph_form_reactions)

In [ ]:
print("len disulph_form_reactions:",len(disulph_form_reactions))
print("len unique_disulph_form_species:",len(unique_disulph_form_species))

In [ ]:
# Dissociation reactions
disso_reactions = react_disso(unique_disulph_form_species)

##### Write output

In [ ]:
file_name = "Prx decamer reactions - no hyperoxidation"

with open(os.path.join(mod_dir,f"{file_name}.txt"), 'w') as f:

    count = 0 # Keep track for pysces reaction name
    species_list = list()
    parameters_list = list()
    dimer_joiner = "__"

    ### Write reactions
    write_reactions.counter = 0
    
    f.write(f"### Reactions\n")
    f.write(f"\n")
    
    # Write decamer association reaction
    f.write("Decamer_disassociation:\n")
    f.write("\tSH_SH__SH_SH__SH_SH__SH_SH__SH_SH = {5}SH_SH\n")
    f.write("\tk_Prx_dec_disasso*SH_SH__SH_SH__SH_SH__SH_SH__SH_SH - k_Prx_asso*SH_SH**exponent\n")
    f.write("\n")
    parameters_list.append("k_Prx_asso")
    parameters_list.append("k_Prx_dec_disasso")
    parameters_list.append("exponent")
    
    # Write sulfe reactions
    write_reactions(set_sulfe_reactions, 
                    "Sulfenilation", 
                    species_list, 
                    "ox", 
                    second_substrate = "H2O2")
    sulfe_reactions_written = write_reactions.counter
    print(f"Sulfenilation reactions written: {sulfe_reactions_written}")
    get_parameters(parameters_list, set_sulfe_reactions)

    # Write disulfide formation reactions
    write_reactions(set_disulph_form_reactions, 
                    "Disulfide bridge formation", 
                    species_list, 
                    "disulf_form")
    disulph_form_reactions_written = write_reactions.counter - sulfe_reactions_written
    print(f"Disulphide bridge formation reactions written: {disulph_form_reactions_written}")
    get_parameters(parameters_list, set_disulph_form_reactions)
    
    # Write dissociation reactions
    reaction_name = "Dissociation"
    f.write(f"# {reaction_name} - decamer\n")
    f.write("\n")
    for disso_reaction in disso_reactions:
        disso_substrate = dimer_joiner.join(
            ["_".join([disso_reaction[0][k],disso_reaction[0][k+1]]) for k in 
                     range(0,len(disso_reaction[0]),2)]
        )
        product_string = ""
        for prod_count,disso_product in enumerate(disso_reaction[1]):
            if prod_count > 0:
                product_string += " + "
            product_string += f"{{{disso_product[0]}}}{disso_product[1]}"
            species_list.append(disso_product[1])

        write_reactions.counter += 1
        f.write(f"R{str(write_reactions.counter)}_disso:\n")
        f.write(f"\t{disso_substrate} > {product_string}\n")
        rate_constant_name = disso_reaction[-1] ###
        if rate_constant_name not in parameters_list:
            parameters_list.append(rate_constant_name)
        f.write(f"\t{rate_constant_name}*{disso_substrate}\n")
        f.write("\n")
    
    Dissociation_reactions_written = write_reactions.counter - \
                                     disulph_form_reactions_written - \
                                     sulfe_reactions_written
    print(f"Dissociation reactions written: {Dissociation_reactions_written}")

    ### Write parameters
    f.write(f"### Parameters\n")
    f.write(f"\n")
    for parameter in parameters_list:
        f.write(f"{parameter} = 0 # add parameter\n")
    f.write("\n")
    
    # Write species
    f.write(f"### Species\n")
    f.write("\n")
    for species in SortedSet(species_list):
        # I only want to write decamer species
        if len(species.split("__")) >= len(original_oligomer)/2: 
            f.write(f"{species} = 0\n")
    print(f"Species written: {len(SortedSet(species_list))}")
    
    ### End
    f.write(f"### End\n")
    f.write(f"\n\n")

In [ ]:
print(f"Sulfe species written: {len(unique_sulfe_species)}")
print(f"Disulph species written: {len(unique_disulph_form_species)}")

## Find reactions for 5 disulphide bridges

In [ ]:
def remove_SS_SS_oligomer(oligomer):
    for dimer_no in range(int(len(oligomer)/2)):
        if oligomer[dimer_no*2] == 'SS' and oligomer[dimer_no*2+1] == 'SS':
            return "None"
    return oligomer

def remove_SS_SS_react(reaction):

    product = reaction[1][0][0]
    for dimer_no in range(int(len(product)/2)):
        if product[dimer_no*2] == 'SS' and product[dimer_no*2+1] == 'SS':
            return "None"
    
    return reaction


In [ ]:
oligomer_length = 10
subunit = "SH"
original_oligomer = [subunit]*oligomer_length

In [ ]:
# Sulfenilation reactions
substrate = "SH"
product = "SOH"
reaction_type = "sulfe"
sulfe_reactions = list()
react_oligomer(original_oligomer, substrate, product, reaction_type, sulfe_reactions, recursive=True)
set_sulfe_reactions = SortedSet(sulfe_reactions)
unique_sulfe_species = find_unique_species(set_sulfe_reactions)

In [ ]:
print("len sulfe_reactions:",len(sulfe_reactions))
print("len unique_sulfe_species:",len(unique_sulfe_species))

In [ ]:
# Disulfide bridge formation reactions round 1
substrate = "SOH"
product = "SS"
reaction_type = "disulf_form"
disulph_form_reactions = list()
for oligomer in unique_sulfe_species: # The unique_sulfi_species contains the unique sulfe species as substrates
    react_oligomer(oligomer, substrate, product, reaction_type, disulph_form_reactions)
set_disulph_form_reactions = SortedSet(disulph_form_reactions)
unique_disulph_form_species = find_unique_species(set_disulph_form_reactions)

In [ ]:
print("len disulph_form_reactions:",len(set_disulph_form_reactions))
print("len unique_disulph_form_species:",len(unique_disulph_form_species))

In [ ]:
# Disulfide bridge formation reactions round 2
substrate = "SOH"
product = "SS"
reaction_type = "disulf_form"

for oligomer in unique_disulph_form_species: # The unique_sulfi_species contains the unique sulfe species as substrates
    react_oligomer(oligomer, substrate, product, reaction_type, disulph_form_reactions)
set_disulph_form_reactions = SortedSet(disulph_form_reactions)
unique_disulph_form_species = find_unique_species(set_disulph_form_reactions)

In [ ]:
print("len disulph_form_reactions:",len(set_disulph_form_reactions))
print("len unique_disulph_form_species:",len(unique_disulph_form_species))

In [ ]:
# Disulfide bridge formation reactions round 3
substrate = "SOH"
product = "SS"
reaction_type = "disulf_form"

for oligomer in unique_disulph_form_species: # The unique_sulfi_species contains the unique sulfe species as substrates
    react_oligomer(oligomer, substrate, product, reaction_type, disulph_form_reactions)
set_disulph_form_reactions = SortedSet(disulph_form_reactions)
unique_disulph_form_species = find_unique_species(set_disulph_form_reactions)

In [ ]:
print("len disulph_form_reactions:",len(set_disulph_form_reactions))
print("len unique_disulph_form_species:",len(unique_disulph_form_species))

In [ ]:
# Disulfide bridge formation reactions round 4
substrate = "SOH"
product = "SS"
reaction_type = "disulf_form"

for oligomer in unique_disulph_form_species: # The unique_sulfi_species contains the unique sulfe species as substrates
    react_oligomer(oligomer, substrate, product, reaction_type, disulph_form_reactions)
set_disulph_form_reactions = SortedSet(disulph_form_reactions)
unique_disulph_form_species = find_unique_species(set_disulph_form_reactions)

In [ ]:
print("len disulph_form_reactions:",len(set_disulph_form_reactions))
print("len unique_disulph_form_species:",len(unique_disulph_form_species))

In [ ]:
# Disulfide bridge formation reactions round 5
substrate = "SOH"
product = "SS"
reaction_type = "disulf_form"

for oligomer in unique_disulph_form_species: # The unique_sulfi_species contains the unique sulfe species as substrates
    react_oligomer(oligomer, substrate, product, reaction_type, disulph_form_reactions)
set_disulph_form_reactions = SortedSet(disulph_form_reactions)
unique_disulph_form_species = find_unique_species(set_disulph_form_reactions)

In [ ]:
print("len disulph_form_reactions:",len(set_disulph_form_reactions))
print("len unique_disulph_form_species:",len(unique_disulph_form_species))

In [ ]:
# Filter out SS_SS reactions and species
set_disulph_form_reactions = [reaction for reaction in list(map(remove_SS_SS_react,set_disulph_form_reactions)) if reaction != "None"]
unique_disulph_form_species = [oligomer for oligomer in list(map(remove_SS_SS_oligomer,unique_disulph_form_species)) if oligomer != "None"]


In [ ]:
print("len disulph_form_reactions:",len(set_disulph_form_reactions))
print("len unique_disulph_form_species:",len(unique_disulph_form_species))

In [ ]:
# Sulfenilation after disulfide bridge formation reactions
substrate = "SH"
product = "SOH"
reaction_type = "sulfe"
sulfe_after_disulph_form_reactions = list()
for oligomer in unique_disulph_form_species: # The unique_sulfi_species contains the unique sulfe species as substrates
    react_oligomer(oligomer, substrate, product, reaction_type, sulfe_after_disulph_form_reactions)
set_sulfe_after_disulph_form_reactions = SortedSet(sulfe_after_disulph_form_reactions)
unique_sulfe_after_disulph_form_species = find_unique_species(set_sulfe_after_disulph_form_reactions)

In [ ]:
# Dissociation reactions
min_SS=1
disso_reactions = react_disso(unique_sulfe_after_disulph_form_species,min_SS=min_SS)

In [ ]:
print("len disso_reactions:",len(disso_reactions))

In [ ]:
file_name = "Prx decamer reactions - no hyperoxidation 5dsb"

with open(os.path.join(mod_dir,f"{file_name}.txt"), 'w') as f:

    count = 0 # Keep track for pysces reaction name
    species_list = list()
    parameters_list = list()
    dimer_joiner = "__"

    ### Write reactions
    write_reactions.counter = 0
    
    f.write(f"### Reactions\n")
    f.write(f"\n")
    
    # Write decamer association reaction
    f.write("Decamer_disassociation:\n")
    f.write("\tSH_SH__SH_SH__SH_SH__SH_SH__SH_SH = {5}SH_SH\n")
    f.write("\tk_Prx_dec_disasso*SH_SH__SH_SH__SH_SH__SH_SH__SH_SH - k_Prx_asso*SH_SH**exponent\n")
    f.write("\n")
    parameters_list.append("k_Prx_asso")
    parameters_list.append("k_Prx_dec_disasso")
    parameters_list.append("exponent")
    
    # Write sulfe reactions
    write_reactions(set_sulfe_reactions, "Sulfenilation", species_list, "ox", second_substrate = "H2O2")
    sulfe_reactions_written = write_reactions.counter
    print(f"Sulfenilation reactions written: {sulfe_reactions_written}")
    get_parameters(parameters_list, set_sulfe_reactions)
    
    # Write sulfe after disulph reactions
    write_reactions(set_sulfe_after_disulph_form_reactions, "Sulfenilation", species_list, "ox", second_substrate = "H2O2")
    sulfe_reactions_after_dsb_written = write_reactions.counter-sulfe_reactions_written
    print(f"Sulfenilation reactions written: {sulfe_reactions_after_dsb_written}")
    get_parameters(parameters_list, set_sulfe_after_disulph_form_reactions)
    
    # Write disulfide formation reactions
    write_reactions(set_disulph_form_reactions, "Disulfide bridge formation", species_list, "disulf_form")
    disulph_form_reactions_written = write_reactions.counter - sulfe_reactions_written-sulfe_reactions_after_dsb_written
    print(f"Disulfide bridge formation reactions written: {disulph_form_reactions_written}")
    get_parameters(parameters_list, set_disulph_form_reactions)
    
    # Write dissociation reactions
    reaction_name = "Dissociation"
    f.write(f"# {reaction_name} - decamer\n")
    f.write("\n")
    for disso_reaction in disso_reactions:
        disso_substrate = dimer_joiner.join(["_".join([disso_reaction[0][k],disso_reaction[0][k+1]])
                                             for k in range(0,len(disso_reaction[0]),2)])
        product_string = ""
        for prod_count,disso_product in enumerate(disso_reaction[1]):
            if prod_count > 0:
                product_string += " + "
            product_string += f"{{{disso_product[0]}}}{disso_product[1]}"
            species_list.append(disso_product[1])

        write_reactions.counter += 1
        f.write(f"R{str(write_reactions.counter)}_disso:\n")
        f.write(f"\t{disso_substrate} > {product_string}\n")
        rate_constant_name = disso_reaction[-1] ###
        if rate_constant_name not in parameters_list:
            parameters_list.append(rate_constant_name)
        f.write(f"\t{rate_constant_name}*{disso_substrate}\n")
        f.write("\n")
    
    Dissociation_reactions_written = write_reactions.counter-disulph_form_reactions_written-sulfe_reactions_written-sulfe_reactions_after_dsb_written
    print(f"Dissociation reactions written: {Dissociation_reactions_written}")

    ### Write parameters
    f.write(f"### Parameters\n")
    f.write(f"\n")
    for parameter in parameters_list:
        f.write(f"{parameter} = 0 # add parameter\n")
    f.write("\n")
    
    # Write species
    f.write(f"### Species\n")
    f.write("\n")
    for species in SortedSet(species_list):
        if len(species.split("__")) >= len(original_oligomer)/2: # I only want to write decamer species
            f.write(f"{species} = 0\n")
            
    print(f"Total reactions written: {write_reactions.counter}")
    print(f"Species written: {len(SortedSet(species_list))}")
    
    ### End
    f.write(f"### End\n")
    f.write(f"\n\n")